---
title: "Full workflow for importing CHIRPS v3 into DHIS2"
short_title: "Import CHIRPS3 data"
---

In this example workflow, we demonstrate how to use DHIS2 Climate Tools to make sure that DHIS2 is continuously updated with the latest climate data. Specifically we show step by step how to **download and import** the latest available **daily precipitation data** from the [CHIRPS v3 data](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview). 

The notebook fetches and imports only data that is not yet present in DHIS2, making it safe to run on a recurring basis and ensuring that DHIS2 stays up to date with the **latest available** climate data. 

If you're only interested in downloading CHIRPS v3 data, see [this detailed step-by-step guide](../../guides/getting-data/climate-hazards-center/chirps3-download.ipynb). 

> Important: This notebook only aggregates to daily periods according the Gregorian calendar. Other calendar systems, like those used in Nepal or Ethiopia, are not yet supported. 

## Prerequisites

Before proceeding with the notebook, make sure the following are in place:

### 1. Required DHIS2 data element

Your DHIS2 instance must contain a data element that can receive the imported data.

For daily precipitation, the data element must have:

- `valueType = NUMBER`
- `aggregationType = SUM`
- It must belong to a data set with `periodType = DAILY`

If this data element does not already exist, you have two options:

- Create the data element manually in DHIS2.
- Or [create the data element using the Python client](../../guides/import-data/prepare-metadata.ipynb).

Once the data element exists, copy its UID and set it as `DHIS2_DATA_ELEMENT_ID` in the **Input Parameters** section further down.

## Library imports

We start by importing the necessary libraries:

In [1]:
from datetime import date
import json

import geopandas as gpd
import xarray as xr
from earthkit import transforms
from metpy.units import units

from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

from dhis2eo.data.chc import chirps3
from dhis2eo.integrations.pandas import dataframe_to_dhis2_json

## Input parameters

Let's first define all the input parameters so they are clearly stated at the top of the notebook. 

For this example we will connect to a public DHIS2 instance, so it's important that you create the precipitation data element (as described previously) and update the `DHIS2_DATA_ELEMENT_ID` below. Note also that the public instance resets every night, so this process will have to be repeated for each new day. 

Note that `IMPORT_FROM_UNITS` and `IMPORT_TO_UNITS` define the units of the source data and the desired units for import. For example, to convert temperature data from Kelvin to degrees Celsius, set these to `K` and `degC`. For other unit conversions, see the `MetPy` documentation for [working with units](https://unidata.github.io/MetPy/latest/tutorials/unit_tutorial.html). 

In [ ]:
# DHIS2 connection
DHIS2_BASE_URL = "https://play.im.dhis2.org/stable-2-42-3-1"
DHIS2_USERNAME = "admin"
DHIS2_PASSWORD = "district"

# DHIS2 import settings
DHIS2_DATA_ELEMENT_ID = '<INSERT-DATA-ELEMENT-ID-HERE>'
DHIS2_ORG_UNIT_LEVEL = 2
DHIS2_DRY_RUN = True                            # default to safe dry-run mode; set to False for actual import

# CHIRPS import configuration
IMPORT_VALUE_COL = "precip"                     # variable name in the downloaded xarray dataset
IMPORT_FROM_UNITS = "mm"                        # units of the original data values
IMPORT_TO_UNITS = "mm"                          # convert to these units before import
IMPORT_START_DATE = "2025-07-01"                # how far back in time to start import
IMPORT_END_DATE = date.today().isoformat()      # automatically tries to import the latest data

# Download settings
DOWNLOAD_FOLDER = "../../guides/data/local"
DOWNLOAD_PREFIX = "chirps3-daily"    # prefix for caching downloads; existing files are reused

# Aggregation settings
TEMPORAL_AGGREGATION = "sum"
SPATIAL_AGGREGATION = "mean"

## Connect to DHIS2

First, we connect the python-client to the DHIS2 instance we want to import into. You can point this to your own instance, but for the purposes of this example we will use one of the public access DHIS2 instances, since these are continuously reset:

In [3]:
# Client configuration
cfg = ClientSettings(
  base_url=DHIS2_BASE_URL,
  username=DHIS2_USERNAME,
  password=DHIS2_PASSWORD
)

client = DHIS2Client(settings=cfg)
info = client.get_system_info()

# Check if everything is working.
# You should see your current DHIS2 version info.
print("Current DHIS2 version:", info["version"])

Current DHIS2 version: 2.42.3.1


## Get the DHIS2 organisation units

In order to download and aggregate the data to our DHIS2 organisation units, we also use the python-client to get the level 2 organisation units from our DHIS2 instance:

In [4]:
### Get org units GeoJSON from DHIS2
org_units_geojson = client.get_org_units_geojson(level=DHIS2_ORG_UNIT_LEVEL)

# Convert GeoJSON to geopandas
org_units = gpd.read_file(json.dumps(org_units_geojson))
org_units

Skipping field groups: unsupported OGR type: 5


,id,code,name,level,parent,parentGraph,geometry
0,O6uvpzGd5pu,OU_264,Bo,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,fdc6uOvgoji,OU_193190,Bombali,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,lc3eMKXaEfw,OU_197385,Bonthe,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,jUb8gELQApl,OU_204856,Kailahun,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,PMa2VCrupOd,OU_211212,Kambia,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,kJq2mPyFEHo,OU_222616,Kenema,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,qhqAxPSTUXp,OU_226213,Koinadugu,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Vth0fbpFcsO,OU_233310,Kono,2,ImspTQPwCqd,ImspTQPwCqd,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,jmIPBj66vD6,OU_246990,Moyamba,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,TEQlaapDQoK,OU_254945,Port Loko,2,ImspTQPwCqd,ImspTQPwCqd,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


## Check when the data was last imported

Since we want to run this script on a regular interval, we want to avoid importing data that has already been imported. We therefore first want to check the last date for which data was imported for the data element we want to import into. This can be done using the convenience function `analytics_latest_period_for_level()` provided by the python-client:

In [5]:
last_imported_response = client.analytics_latest_period_for_level(de_uid=DHIS2_DATA_ELEMENT_ID, level=DHIS2_ORG_UNIT_LEVEL)
last_imported_response

{'meta': {'dataElement': 'UKuEMLLnoQI',
  'level': 2,
  'periodType': 'DAILY',
  'calendar': 'iso8601',
  'years_checked': 31},
 'existing': None,
 'next': None}

Let's extract and report the last imported month:

In [6]:
last_imported_period = last_imported_response["existing"]
last_imported_month_string = last_imported_period["id"][:6] if last_imported_period else None

if last_imported_month_string:
    print(f"Last imported period: {last_imported_month_string}")
else:
    print("No existing data found")

No existing data found


We then use this information to define when we will start the data download, and ensure that we only download data **after** the `last_imported_string`: 

In [7]:
if last_imported_month_string:
    IMPORT_START_DATE_OVERRIDE = max(last_imported_month_string, IMPORT_START_DATE)
else:
    IMPORT_START_DATE_OVERRIDE = IMPORT_START_DATE

print(f'Import will start at {IMPORT_START_DATE_OVERRIDE}')

Import will start at 2025-07-01


## Download the necessary data

In the next step we download all the requested data to the local file system, using convenience functionality from the `dhis2eo.data.chc.chirps3` module. 

Running this step may take some time depending on how many months of data are requested. 

Note that after the initial data download, subsequent runs of this notebook will re-use the previously imported files to avoid repeated downloads of the same data. 

For more details on this step, see our guide for [Downloading CHIRPS v3 data](../../guides/getting-data/climate-hazards-center/chirps3-download.ipynb). 

In [10]:
print(f'Downloading data for the period: {IMPORT_START_DATE_OVERRIDE} to {IMPORT_END_DATE}...')
files = chirps3.daily.download(
    start=IMPORT_START_DATE_OVERRIDE, 
    end=IMPORT_END_DATE, 
    bbox=org_units.total_bounds, 
    dirname=DOWNLOAD_FOLDER, 
    prefix=DOWNLOAD_PREFIX, 
)
files

INFO - 2026-01-14 10:31:01,128 - dhis2eo.data.chc.chirps3.daily - Fetching CHIRPS v3 daily from 2025-7 to 2026-1 (inclusive)
INFO - 2026-01-14 10:31:01,131 - dhis2eo.data.chc.chirps3.daily - Stage/flavor: final/rnl
INFO - 2026-01-14 10:31:01,134 - dhis2eo.data.chc.chirps3.daily - BBox: [-13.3035   6.9176 -10.2658  10.0004]
INFO - 2026-01-14 10:31:01,136 - dhis2eo.data.chc.chirps3.daily - Month 2025-7
INFO - 2026-01-14 10:31:01,141 - dhis2eo.data.chc.chirps3.daily - File already downloaded: C:\Users\karimba\Documents\Github\climate-tools\docs\guides\data\local\chirps3-daily_2025-07.nc
INFO - 2026-01-14 10:31:01,143 - dhis2eo.data.chc.chirps3.daily - Month 2025-8
INFO - 2026-01-14 10:31:01,152 - dhis2eo.data.chc.chirps3.daily - File already downloaded: C:\Users\karimba\Documents\Github\climate-tools\docs\guides\data\local\chirps3-daily_2025-08.nc
INFO - 2026-01-14 10:31:01,155 - dhis2eo.data.chc.chirps3.daily - Month 2025-9
INFO - 2026-01-14 10:31:01,159 - dhis2eo.data.chc.chirps3.daily 

[WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/guides/data/local/chirps3-daily_2025-07.nc'),
 WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/guides/data/local/chirps3-daily_2025-08.nc'),
 WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/guides/data/local/chirps3-daily_2025-09.nc'),
 WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/guides/data/local/chirps3-daily_2025-10.nc'),
 WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/guides/data/local/chirps3-daily_2025-11.nc')]

## Open the downloaded data

Once the data has been downloaded, we can then pass the list of files to `xr.open_mfdataset()`. This allows us to open and work with the data as if it were a single xarray dataset: 

In [12]:
ds_daily = xr.open_mfdataset(files)
ds_daily

<xarray.Dataset> Size: 5MB
Dimensions:      (time: 153, y: 63, x: 62)
Coordinates:
  * time         (time) datetime64[ns] 1kB 2025-07-01 2025-07-02 ... 2025-11-30
  * x            (x) float64 496B -13.32 -13.27 -13.22 ... -10.37 -10.32 -10.27
  * y            (y) float64 504B 10.02 9.975 9.925 9.875 ... 7.025 6.975 6.925
    spatial_ref  int64 8B 0
Data variables:
    precip       (time, y, x) float64 5MB dask.array<chunksize=(31, 63, 62), meta=np.ndarray>
Attributes:
    bbox:     [-13.3035   6.9176 -10.2658  10.0004]
    dataset:  CHIRPS v3 (daily)
    stage:    final
    flavor:   rnl

## Aggregate to organisation units

Since the data is already at the correct daily level, we can proceed directly to aggregate the gridded data to the organisation units from your DHIS2 instance:

In [13]:
print("Aggregating to organisation units...")
ds_org_units = transforms.spatial.reduce(
    ds_daily,
    org_units,
    mask_dim="id",
    how=SPATIAL_AGGREGATION,
)
ds_org_units

Aggregating to organisation units...


<xarray.Dataset> Size: 17kB
Dimensions:      (time: 153, id: 13)
Coordinates:
  * time         (time) datetime64[ns] 1kB 2025-07-01 2025-07-02 ... 2025-11-30
    spatial_ref  int64 8B 0
  * id           (id) object 104B 'O6uvpzGd5pu' 'fdc6uOvgoji' ... 'at6UHUQatSo'
Data variables:
    precip       (id, time) float64 16kB 7.141 5.469 9.233 ... 0.8712 10.88
Attributes:
    bbox:     [-13.3035   6.9176 -10.2658  10.0004]
    dataset:  CHIRPS v3 (daily)
    stage:    final
    flavor:   rnl

## Post-processing

After aggregating the precipitation data to the desired organizational units, we convert the xarray Dataset to a Pandas DataFrame. This makes it easier to inspect the data and prepare it for subsequent post-processing:

In [14]:
dataframe = ds_org_units.to_dataframe().reset_index()
dataframe

,time,id,spatial_ref,precip
0,2025-07-01,O6uvpzGd5pu,0,7.140799
1,2025-07-01,fdc6uOvgoji,0,11.684088
2,2025-07-01,lc3eMKXaEfw,0,4.688370
3,2025-07-01,jUb8gELQApl,0,12.595069
4,2025-07-01,PMa2VCrupOd,0,13.032165
...,...,...,...,...
1984,2025-11-30,jmIPBj66vD6,0,11.706168
1985,2025-11-30,TEQlaapDQoK,0,16.978227
1986,2025-11-30,bL4ooGhyHRQ,0,3.640687
1987,2025-11-30,eIQbndfxQMb,0,19.074561


In this step, we use [MetPy](https://unidata.github.io/MetPy/latest/tutorials/unit_tutorial.html)'s `units` module to automatically convert between units before importing the data. However, in the case of CHIRPS v3, precipitation is already recorded in the desired millimeter import units, so no conversion will be applied: 

In [15]:
if IMPORT_TO_UNITS != IMPORT_FROM_UNITS:
    print(f"Applying unit conversion from {IMPORT_FROM_UNITS} to {IMPORT_TO_UNITS}...")
    # values with source units
    values_with_units = dataframe[IMPORT_VALUE_COL].values * units(IMPORT_FROM_UNITS)
    # convert to target units
    converted = values_with_units.to(IMPORT_TO_UNITS).magnitude
    # update the dataframe
    dataframe[IMPORT_VALUE_COL] = converted
    print(dataframe)
else:
    print("No unit conversion needed")

No unit conversion needed


## Create DHIS2 payload

At this point we have the final data that we want to import into DHIS2. In order to submit the data to DHIS2 we first have to convert the data a standardized JSON format, which can be done with the help of the `dhis2eo` library: 

In [17]:
print(f"Creating payload with {len(dataframe)} values...")
payload = dataframe_to_dhis2_json(
    df=dataframe,
    org_unit_col="id",
    period_col="time",
    value_col=IMPORT_VALUE_COL,
    data_element_id=DHIS2_DATA_ELEMENT_ID,
)
payload['dataValues'][:3]

Creating payload with 1989 values...


[{'orgUnit': 'O6uvpzGd5pu',
  'period': '20250701',
  'value': '7.1407991762',
  'dataElement': 'UKuEMLLnoQI'},
 {'orgUnit': 'fdc6uOvgoji',
  'period': '20250701',
  'value': '11.684088448',
  'dataElement': 'UKuEMLLnoQI'},
 {'orgUnit': 'lc3eMKXaEfw',
  'period': '20250701',
  'value': '4.6883696756',
  'dataElement': 'UKuEMLLnoQI'}]

## Import to DHIS2



In [18]:
print(f"Importing payload into DHIS2 (dryrun={DHIS2_DRY_RUN})...")
res = client.post("/api/dataValueSets", json=payload, params={"dryRun": str(DHIS2_DRY_RUN).lower()})
print(f'Result: {res["response"]["importCount"]}')

Importing payload into DHIS2 (dryrun=True)...
Result: {'imported': 1988, 'updated': 0, 'ignored': 0, 'deleted': 0}


We have now successfully completed a full workflow for downloading, postprocessing, aggegating, and importing daily CHIRPS v3 precipitation data into DHIS2. 